In [ ]:
%pylab nbagg

# Code for measuring basins of attraction and ouput noise robustness of associative memory networks

This code uses Ipython-parallel to train and test the recurrent networks in parallel.
Can be run locally or on a cluster.

## Ipython parallel setup

In [ ]:
#Here it is assumed the the Ipython cluster and engiens are up and running
import ipyparallel as ipp
ipp_client = ipp.Client()
dview = ipp_client.direct_view()
lbview = ipp_client.load_balanced_view()
print('Using {} engines'.format(len(lbview)))

We make sure engines are working from the same directory as our local kernel

In [ ]:
import os
cwd = os.getcwd()
dview['wd'] = cwd

In [ ]:
%%px
import os
os.chdir(wd)

## Local and remote defs for learning and network simulations

In [ ]:
%%px --local

import numpy as np

# functions to find max kappa_in and max kappa_out weights
from max_kappa import sign_constrained_perceptron_max_kappa_out,\
                      sign_constrained_perceptron_max_kappa_in

# functions to train and test the recurrent networks
import associative_memory
# functions from associative_memory module should be available globally
from associative_memory import *


In [ ]:
import asyncio
import ipywidgets

# visualization of parallel jobs progress (optional)
@asyncio.coroutine    
def progress_bar(a):
    w=ipywidgets.FloatProgress()
    display(w)
    while not a.done():
        w.value = (100 * a.progress) / len(a.submitted)
        yield from asyncio.sleep(0.5)
    w.close()

        

# Training

Here we train networks that implements a given set of random memory states as fixed points of the network dynamics. For each memory patterns sets we find the maximal $\kappa_\mathrm{in}$ and maximal $\kappa_\mathrm{out}$ weights.

In [ ]:
#parameters
N=2000 # number of neurons in network
alpha=0.25 # network load P = alpha*N
f_ex=0.3 # activity level of excitatory neurons (in memory states)
f_inh=0.15 # activity level of inhibitory neurons (in memory states)
g_ex=0.8 # fraction of excitatory neurons
n_network=10 # number of networks to train (for averaging over several sets of patterns)

#list of learned networks
networks=[]

#progress visualization
w=ipywidgets.FloatProgress()
display(w)

for n in range(len(networks),n_network):
    w.value = 100. * n / n_network
    
    # Create a set of memories to encode in the network
    X,g = random_patterns_network(N,f_ex,f_inh,g_ex,alpha)

    # Send patterns and neurons type to engiens 
    # (For many engines using dview.push can be inefficient, alternative approaches are to
    # save to disk and load from each engine, or use more efficient comunications (e.g. mpi broadcast)) 
    a=dview.push({'X':X,'g':g},block=True)
    
    # learn max kappa_in weights
    w_kappa_in,c,c1=learn_network(\
            sign_constrained_perceptron_max_kappa_in,N,lbview)

    print('Max kappa_in network: all_conerged={}, memories_are_fixed_points={}'.format(c,c1))

    # learn max kappa_out weights
    w_kappa_out,c,c1=learn_network(\
            sign_constrained_perceptron_max_kappa_out,N,lbview)

    print('Max kappa_out network: all_conerged={}, memories_are_fixed_points={}'.format(c,c1))

    # create learned network dictionary
    networks.append(dict(f_ex=f_ex,f_inh=f_inh,X=X,g=g,w_kappa_in=w_kappa_in,w_kappa_out=w_kappa_out))
    
    # For large networks it might be a good idea to save the networks to disk at this point
w.value = 100.

# Testing

For testing the networks we assume that the `networks` list is defined globally for each engine.

In [ ]:
# Pushing networks to engiens.
# again, for many engines using this can be inefficient. See previouse comment for alternative approches
dview['networks'] = networks

### Stochastic dynamics

In [ ]:

def test_sigma(sigma,n_rep,P_max,n_network,Jij_key,chunksize=10):
    ''' 
    Test the robustness to output noise of magnitude sigma.
    
    # paramaters:
    
    * sigma - magnitude of output noise
    * n_rep - number of times to test each pattern
    * P_max - number of patterns to test from each network
    * n_network - number of networks to test
    * Jij_key - the name of the weights to use for testing (i.e. 'w_kappa_in' or 'w_kappa_out')
    * chunksize - number of jobs to send in each task
    
    # returns
    
    returns an async result object (see usage below)
    
    '''
    #function to run remotely (tests pattern mu from network net_ind)
    # this function assumes `networks` are defined in global scope in each engine
    f = lambda net_ind,mu:\
            test_stochastic_dynamics(sigma,\
                                     networks[net_ind][Jij_key],\
                                     networks[net_ind]['X'][:,mu:mu+1],\
                                     err_thr=0.1)    
    #arguments of f
    net_inds = np.hstack([ind*np.ones(n_rep*P_max,dtype=int) for ind in range(n_network)])
    mu_inds =  np.hstack([(arange(P_max)*ones((n_rep,P_max),dtype=int)).T.flatten()\
                                for ind in range(n_network)])
    
    #remote map
    async_map_result = lbview.map(f,net_inds,mu_inds,block=False,chunksize=chunksize)

    return async_map_result

#parameters
sigma_vec_in = 10**arange(-3.,-1.3,0.04) # values of sigma to test for manx kappa_in solutions
sigma_vec_out = 10**arange(-1.,1,0.04) # values of sigma to test for manx kappa_out solutions
n_rep = 1 # repetitions per patterns
P_max = 100 # patterns per netwokrs
n_network = 10 # number of networks

# submiting jobs
amr_in = [test_sigma(s,n_rep,P_max,n_network,'w_kappa_in',chunksize=10) for s in sigma_vec_in]
amr_out = [test_sigma(s,n_rep,P_max,n_network,'w_kappa_out',chunksize=10) for s in sigma_vec_out]

In [ ]:
# Display progress bars for each task (bar is closed when task is complete) 

loop=asyncio.get_event_loop()
tasks = [asyncio.Task(progress_bar(a)) for a in amr_in] +\
        [asyncio.Task(progress_bar(a)) for a in amr_out]
_=loop.run_until_complete(asyncio.wait(tasks))

In [ ]:
#calculating stable pattarn probability
p_conv_in=[]

for a in amr_in:
    e,t=zip(*a.get())
    p_conv_in.append(np.mean(np.array(t)==500))
    
p_conv_out=[]
for a in amr_out:
    e,t=zip(*a.get())
    p_conv_out.append(np.mean(np.array(t)==500))

#plot results
close()
plot(sigma_vec_in,p_conv_in)
plot(sigma_vec_out,p_conv_out)
xscale('log')
ylim(-0.1,1.1)
grid(1)

### Basins of attraction

In [ ]:
def test_basins(p,n_rep,P_max,n_network,Jij_key,chunksize=10):
    ''' 
    Test the convergence of patterns with initial distortion p.
    
    # paramaters:
    
    * p - magnitude of initial pattern distortion
    * n_rep - number of times to test each pattern
    * P_max - number of patterns to test from each network
    * n_network - number of networks to test
    * Jij_key - the name of the weights to use for testing (i.e. 'w_kappa_in' or 'w_kappa_out')
    * chunksize - number of jobs to send in each task
    
    # returns
    
    returns an async result object (see usage below)
    
    '''
    #function to run remotely (tests pattern mu from network net_ind)
    f = lambda net_ind,mu:\
            test_basin(p,\
                       networks[net_ind][Jij_key],\
                       networks[net_ind]['g'],\
                       networks[net_ind]['f_ex'],\
                       networks[net_ind]['f_inh'],\
                       networks[net_ind]['X'][:,mu:mu+1],\
                       err_thr=0.15)    

    #argumrnts of f
    net_inds = np.hstack([ind*np.ones(n_rep*P_max,dtype=int) for ind in range(n_network)])
    mu_inds =  np.hstack([(arange(P_max)*ones((n_rep,P_max),dtype=int)).T.flatten()\
                                for ind in range(n_network)])
    
    #remote map
    async_map_result = lbview.map(f,net_inds,mu_inds,block=False,chunksize=chunksize)

    return async_map_result

#parameters
p_vec = arange(0.02,0.30,0.01) # values of initial distortion to test
n_rep = 10 # repetitions per patterns
P_max = 10 # patterns per netwokrs
n_network = 10 # number of networks

# submiting jobs
amr_in = [test_basins(p,n_rep,P_max,n_network,'w_kappa_in',chunksize=10) for p in p_vec]
amr_out = [test_basins(p,n_rep,P_max,n_network,'w_kappa_out',chunksize=10) for p in p_vec]


In [ ]:
# Display progress bars for each task (bar is closed when task is complete) 

loop=asyncio.get_event_loop()
tasks = [asyncio.Task(progress_bar(a)) for a in amr_in] +\
        [asyncio.Task(progress_bar(a)) for a in amr_out]
_=loop.run_until_complete(asyncio.wait(tasks))

In [ ]:
# calculate convergence probability

pp_conv_out=[]
for a in amr_out:
    e0,e=zip(*a.get())
    pp_conv_out.append(np.mean(np.array(e)==0.0))

pp_conv_in=[]
for a in amr_in:
    e0,e=zip(*a.get())
    pp_conv_in.append(np.mean(np.array(e)==0.0))

#plot resutls
close()
plot(p_vec,pp_conv_in)
plot(p_vec,pp_conv_out)
ylim(0,1.1)